<a href="https://colab.research.google.com/github/ronykroy/DNN-NLP-and-other-stuff/blob/master/H20_multiClass_newsGroups_AutoML_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# v3: removed the text column in x: predictors

In [2]:
!apt-get install default-jre
!java -version
# install Java.. because.. there is nothing in this colab VM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [3]:
! pip install h2o
# H2O too to be installed SMH

     |████████████████████████████████| 123.6MB 81kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.8-py2.py3-none-any.whl size=123708939 sha256=c0469fb66486f6fa1f1ed0d51c67954d88eea30cd203ff81ee85bc6224a9ce6e
  Stored in directory: /root/.cache/pip/wheels/f7/4c/2f/367de2996c0c2621a9e2e19624be0ea0e4b766df3cd597d916
Successfully built h2o


In [0]:
import h2o

In [5]:
h2o.init(max_mem_size="4G")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpfg6w7wxn
  JVM stdout: /tmp/tmpfg6w7wxn/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpfg6w7wxn/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,7 days and 2 hours
H2O cluster name:,H2O_from_python_unknownUser_i67adb
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [6]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

INFO:sklearn.datasets.twenty_newsgroups:Downloading 20news dataset. This may take a few minutes.
INFO:sklearn.datasets.twenty_newsgroups:Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [0]:
df["Train"] = np.random.rand(len(df)) < 0.8 # get a col full of train boolean 

In [10]:
df.Train.value_counts()

True     8984
False    2330
Name: Train, dtype: int64

In [11]:
reviews = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
train = reviews[reviews["Train"] == "True"]
test = reviews[reviews["Train"] == "False"]

In [52]:
test.shape

(2330, 3)

In [13]:
train.head(2)

label,text,Train
17,"Well i'm not sure about the story nad it did seem biased. What I disagree with is your statement that the U.S. Media is out to ruin Israels reputation. That is rediculous. The U.S. media is the most pro-israeli media in the world. Having lived in Europe I realize that incidences such as the one described in the letter have occured. The U.S. media as a whole seem to try to ignore them. The U.S. is subsidizing Israels existance and the Europeans are not (at least not to the same degree). So I think that might be a reason they report more clearly on the atrocities. What is a shame is that in Austria, daily reports of the inhuman acts commited by Israeli soldiers and the blessing received from the Government makes some of the Holocaust guilt go away. After all, look how the Jews are treating other races when they got power. It is unfortunate.",True
0,"Yeah, do you expect people to read the FAQ, etc. and actually accept hard atheism? No, you need a little leap of faith, Jimmy. Your logic runs out of steam! Jim, Sorry I can't pity you, Jim. And I'm sorry that you have these feelings of denial about the faith you need to get by. Oh well, just pretend that it will all end happily ever after anyway. Maybe if you start a new newsgroup, alt.atheist.hard, you won't be bummin' so much? Bye-Bye, Big Jim. Don't forget your Flintstone's Chewables! :) -- Bake Timmons, III",True


In [0]:
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

In [0]:
# TODO: review how effective this is in removing junk \r \n etc
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [16]:
%%time
# Break reviews into sequence of words
words = tokenize(reviews["text"]) # not sure if there is some sort of leakage here...

CPU times: user 3.6 s, sys: 72.7 ms, total: 3.67 s
Wall time: 18 s


In [17]:
words.head()

C1
well
sure
story
nad
seem
biased
disagree
statement
media
ruin


In [53]:
words.shape

(1195324, 1)

In [18]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

# This takes time to run - left commented out
w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=words) # train on the words thing we just got

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [19]:
# Sanity check - find synonyms for the word 'president'
w2v_model.find_synonyms("president", count = 5) # not bad actually :)

OrderedDict([('yeltsin', 0.839710533618927),
             ('mubarak', 0.8223352432250977),
             ('stephanopoulos', 0.7830407619476318),
             ('miyazawa', 0.7741147875785828),
             ('dole', 0.7653542160987854)])

In [0]:
# Calculate a vector for each review
review_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [21]:
review_vecs

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.100854,0.214922,-0.0053391,-0.124211,0.14442,0.12438,0.0355313,-0.0458748,0.184274,-0.224204,-0.0230852,-0.0970359,0.0635722,0.109533,-0.220632,-0.0662296,-0.119612,0.0475104,0.0277557,0.0519785,-0.0219031,0.16842,-0.0309362,0.0906608,-0.070722,0.0365411,0.00327185,-0.0328439,-0.107563,0.0289882,-0.0894871,0.162283,0.0454493,-0.200985,-0.0213758,0.000782951,0.0256638,0.0313324,-0.094772,-0.194929,0.00821524,-0.0240408,0.107015,-0.0514314,0.0975042,-0.0520109,0.0847034,-0.0814614,0.00743687,-0.0398653,0.0526427,-0.0643699,-0.117195,0.0013163,-0.00257694,0.156557,0.0664971,0.0427845,-0.170556,0.000775099,-0.0425123,-0.132781,0.0695853,0.00850185,0.00435632,0.128878,0.00670197,-0.101024,0.0912273,0.0684253,0.242761,0.0464683,0.141925,-0.0495965,0.00734458,-0.168379,0.0423112,0.037217,0.00871311,0.0339592,-0.0721132,0.129192,0.044666,-0.158464,0.0221064,0.179761,0.0784063,-0.00577739,-0.016471,0.0362513,-0.063352,-0.146119,0.0380497,0.12456,0.133647,0.135837,0.0156643,0.167766,-0.00729291,0.143896
0.0362181,0.164987,0.0729429,-0.0898479,0.020675,0.0149356,0.168858,0.0602448,0.146316,-0.137834,-0.0869655,-0.0830728,0.107615,0.0847902,-0.172672,-0.0970542,-0.0508258,0.0537213,-0.0655368,-0.00169887,0.115304,0.11543,0.0865985,0.119164,-0.101527,0.00956957,0.027875,-0.100589,-0.0403508,0.0486699,-0.00698765,0.0233108,0.00391913,-0.0773472,-0.166856,-0.0381865,0.0985916,0.0127571,-0.262964,-0.0138162,-0.0505553,-0.0135379,0.158761,-0.0785561,-0.00285893,-0.0213931,-0.0409348,-0.0291074,0.0195055,0.00614189,0.0489927,-0.143281,-0.0613934,-0.0128986,0.0436563,0.17227,-0.0457168,-0.0792358,-0.101557,0.00235433,-0.0758121,-0.167949,0.0107867,-0.0459107,-0.00322157,0.0439873,0.00926035,-0.129737,0.155288,0.0457352,0.121833,0.0652034,0.0693419,-0.0421772,-0.0916163,-0.110365,-0.0778806,0.023084,0.0362972,-0.00223716,0.0730762,0.121226,0.144198,-0.210575,-0.0280286,0.0941898,-0.0428202,0.0403882,-0.0723411,0.0437352,-0.041142,-0.0778141,0.0184774,0.114751,0.0819018,0.0146247,-0.0660108,0.217936,0.0627516,0.180888
0.0988381,0.256946,0.0331323,-0.133568,0.115861,0.189089,0.101659,0.0556763,0.129551,-0.224236,-0.0315448,-0.146719,0.0939309,0.0542385,-0.171682,-0.0938371,-0.0941245,0.0197775,0.0226496,0.0273754,0.0535092,0.153357,0.0594746,0.150377,-0.0665057,0.0942774,-0.00140912,-0.0473077,-0.138532,0.109204,-0.0838873,0.174318,0.0802655,-0.148358,-0.0119261,0.0160297,0.0505047,0.0705209,-0.101979,-0.147643,-0.00569729,0.0388345,0.0725605,-0.0523381,0.0364518,-0.0130085,0.0209813,-0.0352256,-0.00112299,-0.0500736,0.101632,-0.0548145,-0.0431246,-0.0128474,0.0311164,0.1855,-0.000722028,-0.0212071,-0.130361,0.0105748,-0.0153413,-0.0759837,0.0499137,-0.0176868,0.0243714,0.125821,0.0269336,-0.0991248,0.137349,0.0703539,0.1991,0.0802391,0.0911705,-0.0636531,-0.0286031,-0.216047,-0.0223152,0.100835,0.0080052,-0.0395069,0.0165128,0.131669,0.0280348,-0.158216,-0.0142503,0.186228,0.103309,0.0362085,-0.0154862,0.10218,-0.0895763,-0.171741,0.039964,0.0251542,0.0439658,0.0952155,-0.0175536,0.122696,0.00151233,0.148789
0.179661,0.0438153,0.0276789,-0.0316723,0.139103,0.148313,0.115865,0.108354,0.0579854,-0.152428,-0.0447966,-0.119619,0.130039,-0.0888611,-0.162869,-0.131771,-0.108584,-0.0081567,-0.200367,-0.0726017,0.0019702,0.196503,-0.0114817,0.15413,-0.0230258,-0.000105817,0.0851186,-0.0818888,0.000278688,0.00708339,-0.0820923,0.0939512,0.0102506,-0.117568,-0.0415668,-0.0826144,0.116524,0.130626,-0.154154,-0.111644,-0.0234984,-0.0554113,0.0570883,-0.0759876,0.0643332,0.0143783,-0.0766481,0.0490793,-0.005918,-0.000355499,0.142617,-0.088382,0.0062889,-0.0391375,0.135717,0

In [0]:
# Add aggregated word embeddings 
ext_reviews = reviews.cbind(review_vecs)
# rename may be... all 'reviews' to 'news'
# ext becaus ewe are extending the reviews df with the vectors

In [0]:
ext_reviews["label_f"]  = ext_reviews["label"] .asfactor()

In [0]:
# Splitting to test train.. along with the review vecs is difficult without the 'Train' boolean column
# hence redoing it from that stage..
# so that we'll be able to do this following cell

In [0]:
ext_train = ext_reviews[ext_reviews["Train"] == "True"]
ext_test = ext_reviews[ext_reviews["Train"] == "True"]

In [0]:
#review_vecs.names
# had a 100 cols
# may be more would give a better model ..?

In [0]:
predictors =  review_vecs.names

In [0]:
response = 'label_f'

In [0]:
from h2o.automl import H2OAutoML

In [0]:
x = ext_train.columns

In [0]:
# x= [x for x in ext_train.columns if x not in ['label', 'Train']]

In [32]:
print(x)

['label', 'text', 'Train', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'C100', 'label_f']


In [0]:
# Identify predictors and response
x = ext_train.columns
x = [x for x in ext_train.columns if x not in ['label', 'Train','text']] # remove columns that can leak data
y = "label_f"
# other_cols = ""
# x.remove(y)

# For binary classification, response should be a factor
#train[y] = train[y].asfactor()
#test[y] = test[y].asfactor()
# Already done

In [34]:
print(x)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'C100', 'label_f']


In [35]:
%%time
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)#, max_runtime_secs =1200)
aml.train(x=x, y=y, training_frame=ext_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 33.8 s, sys: 2.2 s, total: 36 s
Wall time: 1h 47min 34s


In [36]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_AutoML_20191025_033536,0.336151,1.19965,0.604542,0.365471
DRF_1_AutoML_20191025_033536,0.344238,1.83829,0.679328,0.461486
StackedEnsemble_BestOfFamily_AutoML_20191025_033536,0.350187,1.57471,0.718045,0.515589
GBM_5_AutoML_20191025_033536,0.354237,1.31927,0.639196,0.408571
GBM_1_AutoML_20191025_033536,0.358553,1.39241,0.609991,0.372089
GBM_3_AutoML_20191025_033536,0.360415,1.47198,0.61559,0.37895
GBM_2_AutoML_20191025_033536,0.365299,1.42248,0.619295,0.383527
GBM_4_AutoML_20191025_033536,0.377476,1.50195,0.631326,0.398573
DeepLearning_1_AutoML_20191025_033536,0.414942,1.48419,0.656409,0.430873
GLM_grid_1_AutoML_20191025_033536_model_1,0.435804,2.36177,0.895749,0.802366


In [37]:
#aml.leader.confusion_matrix()
aml.leader.cross_validation_metrics_summary

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20191025_033536

No model summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.054410590942894
RMSE: 0.23326077883539273

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.365470999889789
RMSE: 0.6045419752918643


<bound method ModelBase.cross_validation_metrics_summary of >

In [38]:
aml.leader.confusion_matrix(ext_test) # 


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,115,460,Error,Rate
0,376.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025907,10 / 386
1,0.0,408.0,4.0,0.0,14.0,4.0,2.0,13.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.089286,40 / 448
2,0.0,0.0,441.0,0.0,0.0,0.0,1.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039216,18 / 459
3,0.0,0.0,0.0,455.0,0.0,0.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025696,12 / 467
4,0.0,0.0,0.0,0.0,448.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036559,17 / 465
5,0.0,0.0,0.0,0.0,0.0,461.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006466,3 / 464
6,0.0,0.0,0.0,0.0,0.0,0.0,463.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012793,6 / 469
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,483.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004124,2 / 485
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,14.0,458.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031712,15 / 473
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,458.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051760,25 / 483



See the whole table with table.as_data_frame()


In [0]:
con_mat = aml.leader.confusion_matrix(ext_test).as_data_frame()

In [40]:
type(con_mat)

pandas.core.frame.DataFrame

In [41]:
con_mat.index # 23..?

RangeIndex(start=0, stop=23, step=1)

In [42]:
con_mat.iloc[19][19] #

285.0

In [43]:
con_mat.iloc[19][22] #

0.056291390728476824

In [44]:
con_mat.iloc[0:20,0:20]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,376.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,408.0,4.0,0.0,14.0,4.0,2.0,13.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,441.0,0.0,0.0,0.0,1.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,455.0,0.0,0.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,448.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,461.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,463.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,483.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,14.0,458.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,458.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
np.sum(np.diag(con_mat.iloc[0:20,0:20]))/(con_mat.iloc[0:20,0:20].values.sum()) # sum of diagnoal elements/ sum of all in the matrix # 38733477 & SO 53762743

0.9696092619392185

In [0]:
# getting the absolute accuracy  as above

In [47]:
con_mat.iloc[0:20,0:20].values.sum()

8983.0

In [48]:
ext_test.shape # hmmmm

(8984, 104)

In [0]:
#ext_test.head(2)
#ext_test.col_names

In [0]:
con_mat.to_csv('confusion_matrix.csv')

In [51]:
aml.leader.hit_ratio_table().as_data_frame() # SO 53762743

,k,hit_ratio
0,1,0.969609
1,2,0.974285
2,3,0.977736
3,4,0.979294
4,5,0.980630
5,6,0.982077
6,7,0.983970
7,8,0.985083
8,9,0.986196
9,10,0.988089


In [61]:
#train.label.head()
# easier to get pd data and cast it to h2o frame
df.head(2)

,label,text,Train
0,17,Well i'm not sure about the story nad it did seem biased. What\nI ...,True
1,0,"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and...",True


In [68]:
random_news  = h2o.H2OFrame(df.iloc[0:1,0:2])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [70]:
p_words = tokenize(random_news["text"].ascharacter())
p_words.shape

(68, 1)

In [0]:
# how to predict...
p_reviews_vec = w2v_model.transform(p_words, aggregate_method="AVERAGE")

In [79]:
aml.leader.predict(p_reviews_vec)

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p115,p460
17,0.00418369,4.23821e-05,4.49216e-05,4.20622e-05,4.05516e-05,4.2095e-05,4.2274e-05,0.000125282,0.000236057,0.000805391,4.03504e-05,0.000208443,5.3969e-05,5.29381e-05,4.82158e-05,0.000821521,0.00122994,0.988846,0.00135474,0.00173867,0,0


In [0]:
# p17 has the max... so you know it works.. 